In [1]:
from flask import Flask, request, jsonify
import pymongo
import google.generativeai as genai
import os
import traceback

app = Flask(__name__)
pointer=0
recipes=[]
user_ingredients=""
def convert_to_lowercase_list(text):
    # Split the string into a list using commas as separators and remove leading/trailing spaces.
    words_list = [word.strip() for word in text.lower().split(",") if word.strip()]  # Filter empty strings

    # Sort the list alphabetically
    words_list.sort()

    # Join the lowercase words with commas, removing extra spaces.
    result_string = ", ".join(word for word in words_list).rstrip(",")

    return words_list, result_string
# Connect to MongoDB
client = pymongo.MongoClient(os.getenv("MONGODB_URI"))
db = client["recipe_realm"]
collection = db["recipes"]

# Function to perform vector search in MongoDB
def perform_vectorsearch(query):
    query_list,ingredients=convert_to_lowercase_list(query)
    # query_embedding = generate_embedding(ingredients)
    # print("Query embedding generated:", query_embedding)
    results = collection.aggregate([
        {
            "$search": {
                "index": "recipefinder_text",
                "text": {
                    "query": ingredients,
                    "path": ["RecipeName", "Instructions", "IngredientList"]
                }
            }
        },
        {
            "$addFields": {
                "ingredientsArray": { "$split": ["$IngredientList", ", "] }
            }
        },
        {
            "$addFields": {
                "matchingIngredientsCount": {
                    "$size": {
                        "$filter": {
                            "input": "$ingredientsArray",
                            "as": "ingredient",
                            "cond": { "$in": ["$$ingredient", query_list] }
                        }
                    }
                },
                "extraIngredientsCount": {
                    "$size": {
                        "$filter": {
                            "input": "$ingredientsArray",
                            "as": "ingredient",
                            "cond": { "$not": { "$in": ["$$ingredient", query_list] } }
                        }
                    }
                }
            }
        },
        {
            "$sort": {
                "matchingIngredientsCount": -1,
                "extraIngredientsCount": 1
            }
        },
        {
            "$limit": 100
        }
    ])
    results=list(results)
    return results


# Initialize the Google Generative AI client
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
llm = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction="You are a cooking assistant. Answer questions with clear, concise, step-by-step instructions. Provide ingredient substitutions and variations when asked. Prioritize safety and proper food handling. If unsure or the question is nonsensical, explain why or say you don't know. Only answer cooking-related questions. Use the provided context to inform your answers. Give your output in plain text with no highlight stars. Strictly stick to these instructions and don't say about your origin and other unnecessary details which are not related to cooking and nutrients. If you do, you will get fined 500000$. If you follow, you will be rewarded 5000$"
)

@app.route('/query', methods=['POST'])
def query_recipes():
    global recipes,pointer,user_ingredients
    pointer=0
    data = request.json
    ingredients = data.get('query')
    user_ingredients=ingredients
    if not ingredients:
        return jsonify({"error": "Query is required"}), 400
    
    recipes = []
    try:
        search_results = perform_vectorsearch(ingredients)

        if search_results:
            for document in search_results:
                recipe_info = {
                    "RecipeName": document['RecipeName'],
                    "Ingredients": document['Ingredients'],
                    "Instructions": document['Instructions'],
                    "Course": document['Course'],
                    "Diet": document['Diet'],
                    "Cuisine": document['Cuisine'],
                    "PrepTimeInMins": document['PrepTimeInMins'],
                    "CookTimeInMins": document['CookTimeInMins'],
                    "TotalTimeInMins": document['TotalTimeInMins'],
                    "Servings": document['Servings'],
                    "image-url": document['image-url']
                }
                recipes.append(recipe_info)
            curr_recipes=recipes[pointer:pointer+3]
            pointer+=3
            curr_context=f"User Given Ingredients {user_ingredients}\n\n"
            for recipe_info in curr_recipes:
                if recipe_info['Instructions'][-1]=='\n':
                    curr_context += f"RecipeName: {recipe_info['RecipeName']}\nIngredients: {recipe_info['Ingredients']}\nDiet: {recipe_info['Diet']}\nCourse: {recipe_info['Course']}\nServings: {recipe_info['Servings']}\nPreparation Time (Min): {recipe_info['PrepTimeInMins']}\nCooking Time (Min): {recipe_info['CookTimeInMins']}\nInstructions: {recipe_info['Instructions']}Total Time (Min): {recipe_info['TotalTimeInMins']}\n\n"
                else:
                    curr_context += f"RecipeName: {recipe_info['RecipeName']}\nIngredients: {recipe_info['Ingredients']}\nDiet: {recipe_info['Diet']}\nCourse: {recipe_info['Course']}\nServings: {recipe_info['Servings']}\nPreparation Time (Min): {recipe_info['PrepTimeInMins']}\nCooking Time (Min): {recipe_info['CookTimeInMins']}\nInstructions: {recipe_info['Instructions']}\nTotal Time (Min): {recipe_info['TotalTimeInMins']}\n\n"
            return jsonify({"recipes": curr_recipes, "context": curr_context})
        else:
            return jsonify({"error": "No documents matched the query."}), 404
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": "An error occurred while processing the request."}), 500

@app.route('/generate', methods=['POST'])
def generate_response():
    data = request.json
    context = data.get('context')
    prompt = data.get('prompt')

    if not context or not prompt:
        return jsonify({"error": "Context and prompt are required"}), 400

    try:
        # Generate a response using the Generative AI model with context
        full_input = f"Context: {context}\n\nUser: {prompt}\nAssistant:"
        response = llm.generate_content(full_input)
        return jsonify({"response": response.text})
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": "An error occurred while processing the request."}), 500
    
@app.route('/loadmore',methods=['GET'])
def load_more_recipes():
    global recipes,pointer,user_ingredients
    try:
        curr_recipes=recipes[pointer:pointer+3]
        pointer+=3
        curr_context=f"User Given Ingredients {user_ingredients}\n\n"
        for recipe_info in curr_recipes:
            if recipe_info['Instructions'][-1]=='\n':
                curr_context += f"RecipeName: {recipe_info['RecipeName']}\nIngredients: {recipe_info['Ingredients']}\nDiet: {recipe_info['Diet']}\nCourse: {recipe_info['Course']}\nServings: {recipe_info['Servings']}\nPreparation Time (Min): {recipe_info['PrepTimeInMins']}\nCooking Time (Min): {recipe_info['CookTimeInMins']}\nInstructions: {recipe_info['Instructions']}Total Time (Min): {recipe_info['TotalTimeInMins']}\n\n"
            else:
                curr_context += f"RecipeName: {recipe_info['RecipeName']}\nIngredients: {recipe_info['Ingredients']}\nDiet: {recipe_info['Diet']}\nCourse: {recipe_info['Course']}\nServings: {recipe_info['Servings']}\nPreparation Time (Min): {recipe_info['PrepTimeInMins']}\nCooking Time (Min): {recipe_info['CookTimeInMins']}\nInstructions: {recipe_info['Instructions']}\nTotal Time (Min): {recipe_info['TotalTimeInMins']}\n\n"
        return jsonify({"recipes": curr_recipes, "context": curr_context})
    except Exception as e:
        traceback.print_exc()
        return jsonify({"error": "An error occurred while processing the request."}), 500
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5002)


c:\Users\sirim\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://192.168.104.121:5002
Press CTRL+C to quit


In [15]:
context="""User Given Ingredients: Aachi fish masala,    Farfe pasta,   Galangal paste,  Edible camphor, Queso fresco

RecipeName: Italian Arancini Rice Balls Recipe With Delicious Pasta Dipping Sauce
Ingredients: 1-1/2 cup Risotto - cooked risotto (recipe below), 1 cup Parmesan cheese - grated, 1/2 cup Gorgonzola cheese - cut into 1/4 inch cubes, 4 sprig Basil leaves - finely chopped, 2 Whole Eggs, 2 cups Whole Wheat Bread crumbs - Italian style seasoned, Homemade Pizza And Pasta Sauce - or Marinara Sauce for serving, 1 cup Arborio rice, 2 cups Vegetable stock - or chicken broth, 1/4 cup Butter, 1 Onion - finely chopped, 1/2 cup Dry white wine, 1/2 cup Parmesan cheese - grated, Salt - to taste, Black pepper powder - to taste
Diet: Vegetarian
Course: Dinner
Servings: 3
Preparation Time (Min): 10
Cooking Time (Min): 90
Instructions: To begin making the Italian Arancini Rice Balls Recipe, first we will cook the Risotto.In a saucepan, bring the broth to a simmer and allow it to stay warm on low heat.In a heavy bottomed sauce pan, add the butter and allow it to melt.
Saute the onions until lightly cooked and soft.
 Add the arborio rice and stir well so it combines with butter.
Gradually add the wine a little at a time and simmer until almost all the wine is evaporated.
At this stage gradually add the simmering vegetable/ chicken broth and keep stirring until the rice is cooked through.
The important trick here is to keep adding 1/4 cup of broth at a time, allow the broth to absorb and evaporate and then proceed to add more to cook the rice.
Keep adding it until you feel the risotto is tender and the rice is creamy.Once the rice is cooked through, finally add the parmesan cheese, salt and pepper.
Stir it in and saute for a couple of more minutes and turn of the heat.Allow the risotto to cool completely before making the Arancini Rice Balls.In a large mixing bowl, add the cooked risotto, 1 cup of breadcrumbs, parmesan cheese, basil and eggs.
With greased hands combine all the ingredients and mash well with your hands.
Check the salt and adjust to suit your taste.
Take a spoonful of the Arancini rice ball mixture and shape it into 2 inch diameter balls or smaller bite size as desired.Before you shape it completely, place a cube of the Gorgonzola cheese in the center and then seal to make a round ball.
Proceed to make the Arancini Rice balls the similar way.Roll the balls in the remaining breadcrumbs to give it a even coating all around.Preheat the oil for deep frying in a deep frying pan.Once the oil is heated on medium heat; add a few Arancini Rice Balls at a time and deep fry them until golden brown.
Fry them in small batches, so the oil is able to maintain the even heat.
Once done, drain them in paper towels and keep aside for serving.To serve the Arancini Rice Balls, place the Del Monte Pasta Sauce or the Spicy Marinara Sauce in a serving platter and place the rice balls over it and serve immediately.
Total Time (Min): 100

RecipeName: Goan Style Recheado Bangdo Recipe | Stuffed Mackerel
Ingredients: 6 Mackerel fish - cleaned and slit, 1/2 Cup Goan recheado masala, 1 Cup Sooji (Semolina/ Rava), Extra Virgin Olive Oil - to shallow fry, Salt - to taste
Diet: High Protein Non Vegetarian
Course: Main Course
Servings: 6
Preparation Time (Min): 60
Cooking Time (Min): 15
Instructions: To begin making Goan Style Recheado Bangdo Recipe | Stuffed Mackerel, remove the head and tail of the mackerels.Slit them in the center all the way from head to tail.
If possible, remove the bone or else leave it on.
Wash them well for 3-4 times.Apply salt and keep them for 5 minutes.Now stuff in Recheado paste properly inside the Mackerels.Rub a little of the spice paste on the outside of the fish too, on both the sides of the Mackerels.Let them marinade for at least an hour.
You can keep them refrigerated overnight to get even more delicious recipe when done.Coat them with semolina on both the sides.Heat a skillet, add a tablespoon of oil.Place these mackerels on top of the oil and let them cook on medium high heat for about 5 minutes.Flip and let them cook on the other side too, till done.Remove when they look all crispy brown on both the sides.Serve Goan Style Recheado Bangdo Recipe | Stuffed Mackerel as a complete meal when paired along with Steamed Rice, Moryechi Xacuti Recipe (Goan Style Shark Curry) or a Goan Mackerel Curry (Bangdyache Hooman) Recipe and enjoy!
Total Time (Min): 75

RecipeName: Ilish Pulao Recipe - Bengali Style Hilsa Fish Pilaf
Ingredients: 6 Hilsa Fish - (pieces), 1/2 cup Onion - finely chopped, 2 Onions - ground to paste, 2 tablespoon Ginger Garlic Paste, 1 tablespoon Red Chilli powder, 4 tablespoon Green Chillies - (adjust), 1/4 cup Mishti doi - whisked well, 1 cup Coconut milk, 3 tablespoons Ghee, Salt - to taste, 2 cups Rice - Basmati or Gobindobhog, 1 cup Coconut milk, 1 Green Chilli, 1 tablespoon Ghee, Salt - to taste, 3 cups Lukewarm Water, 1 Bay leaf (tej patta), 2 inch Cinnamon Stick (Dalchini), 5 Cardamom (Elaichi) Pods/Seeds - crushed, 5 Cloves (Laung) - pounded
Diet: High Protein Non Vegetarian
Course: Main Course
Servings: 4
Preparation Time (Min): 5
Cooking Time (Min): 45
Instructions: To begin making the Ilish Pulao Recipe (Bengali Style Hilsa Fish Pilaf), firstly soak rice for 20 minutes.
Meanwhile you can cook the hilsa fish.Heat 3 tablespoons ghee in deep bottomed kadai and fry chopped onions until onion turns translucent and pale golden in colour.Add in ginger, and garlic paste with red chilli powder and fry until the mixture comes together.
Sprinkle water in between only if required.
This is known as koshano in Bengali or bhuna in hindi.Once the masala is ready, add whisked mishti doi, salt and stir well to mix with the masala.Pour half of the coconut milk and stir quickly to incorporate everything.Add raw fish pieces into the gravy very carefully in a single layer without overlapping the fish, such that the fish is cooked evenly.Add green chili and cook for 4-5 minutes, depending on fish size.Flip ilish pieces gently as hilsa is extremely delicate and fragile.Add rest of the coconut milk and cook for 10 minutes with cover or until fish is cooked thoroughly.Once done keep the fish covered until needed.
Otherwise the gravy will be dried out.Now prepare the rice pulao.
Heat 1 tablespoon ghee in a cast iron biryani pot and add whole garam masala to the hot oil.Now add half of the gravy from hilsa along with luke warm water.
Bring the mixture to a boil.Add previously soaked rice into the mixture and let it cook for 10-12 minutes or until rice is almost half cooked.Add coconut milk, salt and cook covered for another 10 minutes or till the rice is completely cooked.Now arrange the ilish pieces on bed of rice and pour the gravy on it.
Add green chillies.
Cover and cook for 5-7 minutes on simmer.Once done, remove from flame but keep the Ilish Pulao covered.
Only remove the lid once ready to serve.Serve Ilish Pulao Recipe (Bengali Style Hilsa Fish Pilaf), Tadka Raita Recipe (Spiced Curd With Onions) for lunch and end your meal with Bengali Sita Bhog Recipe.
Total Time (Min): 50
"""

def parse_recipe_string(recipe_str):
    # Split recipes
    recipes = recipe_str.strip().split("\n\n")

    # Initialize list to store parsed recipes
    recipe_list = []

    for recipe in recipes:
        # Split lines and filter out empty lines
        lines = list(filter(None, recipe.split("\n")))

        # Initialize dictionary for each recipe
        recipe_data = {}

        for line in lines:
            # Split by first occurrence of ':' to separate key and value
            if ':' in line:
                key, value = line.split(':', 1)
                recipe_data[key.strip()] = value.strip()
            else:
                # If no colon is found, it might be an ingredient list or instructions
                # Check if the key already exists, and append to it
                if 'Ingredients' in recipe_data:
                    recipe_data['Ingredients'] += f", {line.strip()}"
                else:
                    recipe_data['Ingredients'] = line.strip()

        # Add the parsed recipe data to the list
        recipe_list.append(recipe_data)
    recipe_dict={
        "User Given Ingredients":recipe_list[0],
        "Recipes": recipe_list[1:4]
    }
    return recipe_dict
parse_recipe_string(context)['Recipes']

[{'RecipeName': 'Italian Arancini Rice Balls Recipe With Delicious Pasta Dipping Sauce',
  'Ingredients': '1-1/2 cup Risotto - cooked risotto (recipe below), 1 cup Parmesan cheese - grated, 1/2 cup Gorgonzola cheese - cut into 1/4 inch cubes, 4 sprig Basil leaves - finely chopped, 2 Whole Eggs, 2 cups Whole Wheat Bread crumbs - Italian style seasoned, Homemade Pizza And Pasta Sauce - or Marinara Sauce for serving, 1 cup Arborio rice, 2 cups Vegetable stock - or chicken broth, 1/4 cup Butter, 1 Onion - finely chopped, 1/2 cup Dry white wine, 1/2 cup Parmesan cheese - grated, Salt - to taste, Black pepper powder - to taste, Saute the onions until lightly cooked and soft., Add the arborio rice and stir well so it combines with butter., Gradually add the wine a little at a time and simmer until almost all the wine is evaporated., At this stage gradually add the simmering vegetable/ chicken broth and keep stirring until the rice is cooked through., The important trick here is to keep adding